In [ ]:
import tvm
from tvm import relay
import tvm.relay.analysis_tools

We'll start by examining a simple Relay program:

In [ ]:
program = relay.const(1) - (relay.var('x') * relay.var('y'))

This simple analysis pass visits all `Call`s. It uses the `AnalysisPass` helper method `_add_detail` to attach analysis results to an expression. In this case, it attaches an analysis result named `'readable_name'` to the `Call` being visited. `_add_detail` is one of the main conveniences added by this simple analysis framework.

In [ ]:
class GetReadableName(relay.analysis_tools.AnalysisPass):
    def visit_call(self, call):
        super().visit_call(call)
        self._add_detail(call, readable_name=call.op.name)

We can build another simple `AnalysisPass` to give all of our nodes unique ids:

In [ ]:
class GetIndex(relay.analysis_tools.AnalysisPass):
    def __init__(self):
        super().__init__()
        self.__id = 0

    def visit_call(self, call):
        super().visit_call(call)
        self._add_detail(call, id=self.__id)
        self.__id += 1

Next, we use the `run_analyses` helper method to run a batch of analyses on our program.

In [ ]:
analyses = [GetReadableName(), GetIndex()]
analysis_results, summary_results = relay.analysis_tools.run_analyses(program, analyses)
analysis_results

As we can see, the analysis results are in a bit of a raw form. The rest of the interesting functions in the analysis framework mostly pertain to helping wrangle the output into a useful form.

For example, we can get the columns from the output, and then once we have the columns, we can turn the data into record format:

In [ ]:
analysis_columns = relay.analysis_tools.get_analysis_columns(analysis_results)
analysis_columns

In [ ]:
records = relay.analysis_tools.get_records(analysis_results, analysis_columns)
records

Record format helps us put the data into pandas:

In [ ]:
import pandas as pd
pd.DataFrame.from_records(records, columns=['id', 'op'], index='id')

Now, let's see what it looks like with a bigger program, like Mobilenet:

In [ ]:
from tvm.relay.testing.mobilenet import get_workload

module, params = get_workload()

analyses = [GetReadableName(), GetIndex()]
analysis_results, summary_results = relay.analysis_tools.run_analyses(module['main'], analyses)
records = relay.analysis_tools.get_records(analysis_results, analysis_columns)
pd.DataFrame.from_records(records, columns=['id', 'op'], index='id')

## Building Analysis Summaries

The last feature of the analysis framework is the ability to build _summaries_, which are simply useful ways to add analysis results which are not tied to an expression within the program, but to the program as a whole.

We can make a summary by overriding `_summarize`. This method runs once the `AnalysisPass` finishes visiting all expressions in a program. `_summarize` can access the `_existing_data` field to access previously-generated data. Using this data, it can then generate summary info and attach it to the program using `_add_summary`.

Here, we show a summary pass which generates a histogram of call types:

In [ ]:
class SummarizeOpTypes(relay.analysis_tools.AnalysisPass):
    def _summarize(self):
        histogram = {}
        for node, data in self._existing_data.items():
            if data['readable_name'] not in histogram:
                histogram[data['readable_name']] = 1
            else:
                histogram[data['readable_name']] += 1
        self._add_summary(histogram)

We generate summaries with `run_analyses` as well: 

In [ ]:
analyses = [GetReadableName(), GetIndex(), SummarizeOpTypes()]
analysis_results, summary_results = relay.analysis_tools.run_analyses(module['main'], analyses)
summary_results

There are also utilities for pulling out information about summaries:

In [ ]:
summary_columns = relay.analysis_tools.get_summary_columns(summary_results)
summary_columns

We can generate summaries for two networks, and display the results in a summary table:

In [ ]:
from tvm.relay.testing.resnet import get_workload

module, params = get_workload()

mobilenet_summary = summary_results

analyses = [GetReadableName(), GetIndex(), SummarizeOpTypes()]
_, resnet_summary = relay.analysis_tools.run_analyses(module['main'], analyses)
resnet_columns = relay.analysis_tools.get_summary_columns(resnet_summary)

summary_columns.update(resnet_columns)
summary_columns

In [ ]:
mobilenet_record = relay.analysis_tools.summary_to_record(summary_columns, mobilenet_summary)
resnet_record = relay.analysis_tools.summary_to_record(summary_columns, resnet_summary)

pd.DataFrame.from_records([mobilenet_record, resnet_record], columns = summary_columns).fillna(0).astype('int')